In [1]:
import os
import ast
import re
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Summarize baseline results into Pandas DataFrame

In [2]:
def summarize(path, fname):
    model_meta = pd.read_parquet(os.path.join(path, fname))
    def replace(x):
        try:
            return ast.literal_eval(x)
        except:
            return x
    for col in ['train_loss', 'train_acc', 'valid_loss', 'valid_acc']:
        try:
            model_meta[col] = model_meta[col].apply(replace)
        except:
            pass

    model_meta['epochs'] = model_meta['train_loss'].apply(len)
    model_meta['final_train_loss'] = model_meta['train_loss'].apply(lambda x: x[-1])
    model_meta['final_valid_acc'] = model_meta['valid_acc'].apply(lambda x: x[-1])
    model_meta['final_valid_loss'] = model_meta['valid_loss'].apply(lambda x: x[-1])
    model_meta['best_valid_loss'] = model_meta['valid_loss'].apply(min)

    return model_meta

In [3]:
model_meta = pd.DataFrame()

root_dir = ''
source_train_baseline_dir ='baseline_reverse_20k'

for f in os.listdir(os.path.join(root_dir, source_train_baseline_dir)):
    if f.endswith('.parquet'):
        trial_results = summarize(os.path.join(root_dir, source_train_baseline_dir), f)
        _, iter_idx, _ = re.split('[_.]', f)
        trial_results.loc[:, 'iter_idx'] = iter_idx

        try:
            with open(os.path.join(root_dir, source_train_baseline_dir, f'args_{iter_idx}.json')) as arg_f:
                args = json.load(arg_f)
        except OSError:
            print('No corresponding args.json file associated with trial!')

        for key in ['num_epochs', 'lr', 'batch_size',
                    'train_seed', 'hidden_size', 'data_sampler_seed']:
            trial_results.loc[:, key] = args[key]

        model_meta  = model_meta.append(trial_results, ignore_index=True)

# Summarize performance for trained models

In [4]:
performance_cols = ['mimic_test_loss', 'mimic_test_acc', 'mimic_test_auc',
                    'chexpert_test_loss', 'chexpert_test_acc', 'chexpert_test_auc']

In [5]:
model_meta.max()[performance_cols]

mimic_test_loss       0.595955
mimic_test_acc        0.830033
mimic_test_auc        0.895857
chexpert_test_loss    0.683538
chexpert_test_acc     0.755875
chexpert_test_auc     0.860108
dtype: object

In [6]:
model_meta.mean()[performance_cols]

mimic_test_loss       0.594662
mimic_test_acc        0.829155
mimic_test_auc        0.894722
chexpert_test_loss    0.666971
chexpert_test_acc     0.737167
chexpert_test_auc     0.854236
dtype: float64

In [7]:
model_meta.std()[performance_cols]

mimic_test_loss       0.001213
mimic_test_acc        0.001209
mimic_test_auc        0.001300
chexpert_test_loss    0.015306
chexpert_test_acc     0.020890
chexpert_test_auc     0.006227
dtype: float64